In [23]:
from pandas import Timestamp
import numpy as np
from numba import njit
import numba
# import pycurl_requests as requests
import requests
import pandas as pd
from pprint import pprint
import sys
import itertools
import snoop
sys.path.append(
    "/Users/GlebSokolov/.vscode/extensions/almenon.arepl-2.0.3/node_modules/arepl-backend/python"
)
# from arepl_dump import dump
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"


In [2]:
import coinbase
from coinbase.wallet.model import APIObject
from coinbase.wallet.client import Client
import cbpro

coinbase_API_key = "PZsUB9TdWedNjJcu"
coinbase_API_secret = "UP8Jz2cfXvDRdgeeNwFy6fCxfJCkQFea"
client = Client(coinbase_API_key, coinbase_API_secret)


In [19]:
import time
import numpy as np

symbol = "AGLD-USDT"

since = "2021-10-05 10:00:00"
end = "2021-10-05 11:00:00"
since_ts = Timestamp(since).timestamp()
end_ts = Timestamp(end).timestamp()


In [4]:
%%capture
client.get_spot_price(currency='AGLD')
client._make_api_object(client._get('v2', 'prices', 'AGLD-USD', 'historic'), APIObject)

In [5]:
%%capture
import cbpro
public_client = cbpro.PublicClient()
public_client.get_product_ticker(product_id='AGLD-USD')
responses_ = list(public_client.get_product_trades(product_id=symbol)
)

In [6]:
# %%snoop
trade_ids = np.zeros(len(responses_), dtype=int)
rrr = np.zeros(len(responses_), dtype=np.int64)
for i in range(rrr.shape[0]):
    rrr[i] = int(Timestamp(responses_[i]["time"]).timestamp())
    trade_ids[i] = responses_[i]["trade_id"]


In [20]:
(rrr[1] - end_ts)/(3600*24)

79.65215277777777

In [8]:
# @njit
# %%snoop
def findid(lst, ts_start, ts_end, trade_ids):
    ids = []
    for i in range(len(lst)):
        if ts_start <= lst[i] <= ts_end:
            ids.append((trade_ids[i], i))
    return np.array(ids)

In [21]:
findid(rrr, since_ts, end_ts, trade_ids)

array([[15582, 73934],
       [15581, 73935],
       [15580, 73936],
       [15579, 73937],
       [15578, 73938],
       [15577, 73939],
       [15576, 73940],
       [15575, 73941],
       [15574, 73942],
       [15573, 73943],
       [15572, 73944],
       [15571, 73945],
       [15570, 73946],
       [15569, 73947],
       [15568, 73948],
       [15567, 73949],
       [15566, 73950],
       [15565, 73951],
       [15564, 73952],
       [15563, 73953],
       [15562, 73954],
       [15561, 73955],
       [15560, 73956],
       [15559, 73957],
       [15558, 73958],
       [15557, 73959],
       [15556, 73960],
       [15555, 73961],
       [15554, 73962],
       [15553, 73963],
       [15552, 73964],
       [15551, 73965],
       [15550, 73966],
       [15549, 73967],
       [15548, 73968],
       [15547, 73969],
       [15546, 73970],
       [15545, 73971],
       [15544, 73972],
       [15543, 73973],
       [15542, 73974],
       [15541, 73975],
       [15540, 73976],
       [155

In [17]:
data = public_client.get_product_historic_rates(
    start=since, end=end, product_id="AGLD-USDT"
)
data = pd.DataFrame(data, columns=["time", "low", "high", "open", "close", "volume"])
data["time_iso"] = pd.to_datetime(data["time"], unit="s")


In [18]:
r = requests.get('https://api.binance.com/api/v3/aggTrades',
        params = {
            "symbol" : 'AGLDUSDT',
            "startTime": since_ts,
            "endTime": end_ts
    }).json()

In [27]:
# %%script false --no-raise-error
# %%snoop
# headers = {
#     "User-Agent": "python-requests/2.26.0",
#     "Accept": "*/*",
#     "Connection": "keep-alive",
# }
responses = [
    requests.get(f"https://api.exchange.coinbase.com/products/AGLD-USDT/trades?limit=1000&after={i*1000}").json() for i in range(70)
]
responses.pop(0)

{'message': 'Invalid positive integer value for parameter - after'}

In [26]:
%%script false --no-raise-error
# dir(public_client)

object_methods = [
    method_name
    for method_name in dir(public_client)
    if callable(getattr(public_client, method_name))
]
pprint(object_methods)
